In [ ]:
import threading
import queue
import time

q = queue.Queue()

class State:
    items_produced = 0 # only set by producer
    items_comsumed = 0 # only set by consumer
    done_producing = False # only set by producer

state = State()

def producer(state: State):
    for i in range(5):
        q.put(i)
        print(f"Produced {i}")
        state.items_produced = state.items_produced + 1
        time.sleep(1)
    state.done_producing = True
    
def consumer(state: State):
    # loop always
    while True:
        # check if item
        items_available =  state.items_produced - state.items_comsumed > 0
        if items_available:
            # get item
            item = q.get(block=False) # avoid exceptions (= slow)
            state.items_comsumed = state.items_comsumed + 1
            q.task_done()
            # print
            print(f"Consumed {item}")
        # stop when consumed all
        if state.done_producing and state.items_produced == state.items_comsumed:
            break
            
def run():
    producer_thread = threading.Thread(target=producer, args=[state])
    consumer_thread = threading.Thread(target=consumer, args=[state])

    producer_thread.start()
    consumer_thread.start()

    producer_thread.join()  # Wait for producer to finish
    q.join()                # Wait for all tasks to be done
    consumer_thread.join()  # Wait for consumer to finish
    
run()

The `queue.Queue` class is a thread-safe implementation of a multi-producer, multi-consumer queue, which is especially useful for thread communication and data exchange.

**Usage**: Threads can safely put items to and get items from the queue without needing additional synchronization. The queue manages all the necessary locking internally.